# Ratings and Revenue

Is there a noticeable relationship between ratings and the top film gross revenue?


## import the data

In [1]:
#necessary imports

import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [2]:
#converts tsv files

tsv_file = './zippedData/rt.movie_info.tsv.gz'
convert = pd.read_table(tsv_file, delimiter='\t')
convert.to_csv('./zippedData/rt.movie_info.csv.gz')

tsv_file_r = './zippedData/rt.reviews.tsv.gz'
convert_r = pd.read_table(tsv_file_r, delimiter='\t', encoding = 'unicode_escape')
convert_r.to_csv('./zippedData/rt.reviews.csv.gz')

In [3]:
#importing data using glob

datafiles = glob("./zippedData/*.csv.gz")
datafiles

['./zippedData/rt.reviews.csv.gz',
 './zippedData/imdb.title.crew.csv.gz',
 './zippedData/rt.movie_info.csv.gz',
 './zippedData/tmdb.movies.csv.gz',
 './zippedData/imdb.title.akas.csv.gz',
 './zippedData/imdb.name.basics.csv.gz',
 './zippedData/imdb.title.basics.csv.gz',
 './zippedData/tn.movie_budgets.csv.gz',
 './zippedData/bom.movie_gross.csv.gz',
 './zippedData/imdb.title.principals.csv.gz']

In [4]:
datafiles_dict = {}
for filename in datafiles:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".gz", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename)
    datafiles_dict[filename_cleaned] = filename_df

In [5]:
print(datafiles_dict.keys())

dict_keys(['rt_reviews', 'imdb_title_crew', 'rt_movie_info', 'tmdb_movies', 'imdb_title_akas', 'imdb_name_basics', 'imdb_title_basics', 'tn_movie_budgets', 'bom_movie_gross', 'imdb_title_principals'])


## Ratings

Review user-generated review/rating aka Audience Score

### Of the films with the top 5 box office revenue, how were the ratings?

In [51]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT rating, box_office
FROM rt_reviews
JOIN rt_movie_info
USING('pid')  
ORDER BY box_office DESC;""")

pd.set_option('display.max_columns', None)
dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())
dft.head()

,rating,box_office
0,3/4,368000000.0
1,2.5/5,303001229.0
2,7,299300000.0
3,3.5/4,279167575.0
4,B-,261835892.0


**Finding**: We can see that there is not a significant with the top 5 films having the highest box office revenue.

### Is there a common genre associated with top ratings?

In [58]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT rating, box_office, genre_action, genre_kids, genre_adventure, genre_animation, genre_classics, 
genre_comedy, genre_documentary, genre_drama, 
genre_horror, genre_international, genre_kids, 
genre_muscial, genre_mystery, genre_scifi, 
genre_spirituality, genre_sports, genre_romance,
genre_television, genre_western
FROM rt_reviews
JOIN rt_movie_info
USING('pid')  
WHERE (rating == '4/4') OR (rating == 'A') OR (rating == '10') OR (rating == '5/5') 
ORDER BY box_office DESC;""")

pd.set_option('display.max_columns', None)
dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())
dft.head()

,rating,box_office,genre_action,genre_kids,genre_adventure,genre_animation,genre_classics,genre_comedy,genre_documentary,genre_drama,genre_horror,genre_international,genre_kids,genre_muscial,genre_mystery,genre_scifi,genre_spirituality,genre_sports,genre_romance,genre_television,genre_western
0,A,71844424.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,5/5,54983983.0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,A,36800000.0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,A,32700000.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5/5,31155435.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


**Finding**: Films with the highest ratings and highest box office revenue are primarily distributed within the action, adventure, comedy and drama genres.

### Popularity, Gross Profit and Net Revenue: are there any notable patterns?

In [ ]:
code here

**Finding**: add conculsion here

## Data Visualizations 

line graph of what gross profit looks like considering popularity. (popularity is the Y axis and dollars is the X axis)

In [59]:
code here

SyntaxError: invalid syntax (<ipython-input-59-f206dc6fcd6f>, line 1)